In [1]:
from dataclasses import dataclass
from pathlib import Path

In [2]:
@dataclass
class PrepareInputOutputData:
    processed_image_folder : Path
    blur_image_folder : Path

@dataclass
class ModelParametersConfig:
    input_shape : tuple
    batch_size : int
    kernel_size : int
    latent_dim : int
    layer_filters : list

@dataclass
class TrainingModelConfig:
    model_dir : Path
    HDFmodel_path : Path
    model_path : Path
    loss:str
    optimizer: str
    metrics:list
    batch_size: int
    epochs:int
    model_history_path :Path


In [3]:
from autoencoder.constants import filepath
from autoencoder import logger
from autoencoder.utils.util_functions import create_dir, read_yaml

In [4]:
class ConfigurationManager():
    def __init__(self,
                 config_filepath = filepath.CONFIG_FILE_PATH,
                 params_filepath = filepath.PARAMS_FILE_PATH,
                 secret_filepath = filepath.SECRET_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.secret = read_yaml(secret_filepath)
    
    def get_data_preparation_config(self) -> PrepareInputOutputData:
        config = self.config.data_paths

        input_output_data_config = PrepareInputOutputData(
            processed_image_folder=config.processed_image_folder,
            blur_image_folder= config.blur_image_folder
        )

        return input_output_data_config
    
    def get_model_param_config(self) -> ModelParametersConfig:
        parameters = self.params

        model_param_config = ModelParametersConfig(
            input_shape=tuple(parameters.input_shape),
            batch_size= parameters.batch_size,
            kernel_size= parameters.kernel_size,
            latent_dim=parameters.latent_dim,
            layer_filters=  parameters.layer_filters
        )

        return model_param_config
    
    def get_model_training_config(self) -> TrainingModelConfig:
        config = self.config.model_paths
        params = self.params

        model_training_config = TrainingModelConfig(
            model_dir= config.model_dir,
            HDFmodel_path=config.HDFmodel_path,
            model_path=config.model_path,
            loss=params.loss,
            optimizer=params.optimizer,
            metrics=params.metrics,
            batch_size=params.batch_size,
            epochs=params.epochs,
            model_history_path=config.model_history_path

        )

        return model_training_config
    


In [5]:
#import Model Training
import numpy as np
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras import Model, Input, regularizers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing import image

import glob
from tqdm import tqdm
import warnings;
warnings.filterwarnings('ignore')



from keras.layers import Dense, Input
from keras.layers import Conv2D, Flatten
from keras.layers import Reshape, Conv2DTranspose, BatchNormalization, Add
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import backend as K

%matplotlib inline
import random
import os
import tensorflow as tf
from tqdm import tqdm

In [6]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers, models, Input
from skimage.metrics import peak_signal_noise_ratio as psnr

In [7]:
class ModelBuilding():
    def __init__(self, params_config = ModelParametersConfig):
        self.params_config = params_config

    def residual_block(self, x, filters, kernel_size, stride=1):
        shortcut = x
        x = layers.Conv2D(filters, kernel_size, strides=stride, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        x = layers.Conv2D(filters, kernel_size, strides=stride, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.add([shortcut, x])
        x = layers.ReLU()(x)
        return x
    
    def build_autoencoder(self) -> tf.keras.Model:
        input_shape = self.params_config.input_shape
        kernel_size = self.params_config.kernel_size
        latent_dim = self.params_config.latent_dim
        layer_filters = self.params_config.layer_filters
        logger.info(f'The Input shape is {input_shape}')
        logger.info(f'The kernel size is {kernel_size} ')
        logger.info(f'The latent vector Lenght is {latent_dim}')
        logger.info("Building Encoder Part of the Model")
        inputs = Input(shape=input_shape)
        x = layers.Conv2D(layer_filters[0], (kernel_size, kernel_size), padding='same', strides=2)(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)

        x = self.residual_block(x, layer_filters[0], kernel_size)
        x = layers.Conv2D(layer_filters[1], (kernel_size, kernel_size), padding='same', strides=2)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)

        x = self.residual_block(x, layer_filters[1], kernel_size)
        x = layers.Conv2D(layer_filters[2], (kernel_size, kernel_size), padding='same', strides=2)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)

        x = self.residual_block(x, layer_filters[2], kernel_size)

        # Flatten and Latent Vector
        x = layers.Flatten()(x)
        latent = layers.Dense(latent_dim, name='latent_vector')(x)

        # Decoder
        logger.info("Building Dencoder Part of the Model")
        x = layers.Dense(16 * 16 * layer_filters[2])(latent)  # Adjust dimensions to match encoder output shape
        x = layers.Reshape((16, 16, layer_filters[2]))(x)
        
        x = self.residual_block(x, layer_filters[2], kernel_size)
        x = layers.Conv2DTranspose(layer_filters[1], (kernel_size, kernel_size), padding='same', strides=2)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)

        x = self.residual_block(x, layer_filters[1], kernel_size)
        x = layers.Conv2DTranspose(layer_filters[0], (kernel_size, kernel_size), padding='same', strides=2)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)

        x = self.residual_block(x, layer_filters[0], kernel_size)
        x = layers.Conv2DTranspose(32, (kernel_size, kernel_size), padding='same', strides=2)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)

        x = self.residual_block(x, 32, kernel_size)

        # Output layer
        outputs = layers.Conv2D(3, (kernel_size, kernel_size), activation='sigmoid', padding='same')(x)

        # Autoencoder model
        autoencoder = models.Model(inputs, outputs)
        logger.info("Autoencoder Model Building Process Completed")
        return autoencoder
    
    
    

In [8]:
class PrepareDataForTraining():
    def __init__(self, model_config:PrepareInputOutputData):
        self.model_config = model_config

    def image_to_array(self, folder_path) -> np.ndarray:
        array = []
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            try:
                img = image.load_img(img_path)
                img = image.img_to_array(img)
                img = img/255.0
                array.append(img)
            except Exception as e:
                raise e
        logger.info("Image Data Successfully Converted into numpy array")
        return np.array(array)
    
    def prepareinputdata(self)->np.ndarray:
        try:
            logger.info("Converting Blur Image Data into Array for Model Input")
            input_data_path = self.model_config.blur_image_folder
            input_data = self.image_to_array(input_data_path)
        except Exception as e:
            raise e
        return input_data

    def prepareoutputdata(self)->np.ndarray:
        try:
            logger.info("Converting Clean Image Data into Array for Model Output")
            output_data_path = self.model_config.blur_image_folder
            output_data = self.image_to_array(output_data_path)
        except Exception as e:
            raise e
        return output_data

In [9]:
from autoencoder.utils.util_functions import create_dir
import json

In [12]:
class TrainModel():
    def __init__(self, train_config = TrainingModelConfig):
        self.train_config = train_config

    def train_model(self, x : np.ndarray, y : np.ndarray, model : tf.keras.Model):
        logger.info(f"Compiling Autoencoder Model with loss : {self.train_config.loss}")
        logger.info(f"Compiling Autoencoder Model with optimizer : {self.train_config.optimizer}")
        logger.info(f"Compiling Autoencoder Model with metrics : {self.train_config.metrics}")
        model.compile(loss=self.train_config.loss, optimizer = self.train_config.optimizer, metrics=self.train_config.metrics)
        logger.info("Model Compiled Successfully")
        lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               verbose=1,
                               min_lr=0.5e-6)
        callbacks = [lr_reducer]
        logger.info(f'Training Model for Epoch : {self.train_config.epochs}')
        logger.info(f'Training Model with BatchSize : {self.train_config.batch_size}')
        history = model.fit(x, y, epochs=self.train_config.epochs,batch_size=self.train_config.batch_size, callbacks = callbacks)
        logger.info('Model Trained Successfully Now saving model') 
        create_dir([self.train_config.model_dir])
        model.export(self.train_config.model_path)
        model.save(self.train_config.HDFmodel_path)
        with open(self.train_config.model_history_path, 'w') as f:
            json.dump(history.history, f)
        logger.info(f'model saved at {self.train_config.model_path}')



In [13]:
try:
    config_manager = ConfigurationManager()
    data_preparation_config = config_manager.get_data_preparation_config()
    preparedata = PrepareDataForTraining(data_preparation_config)
    input_data = preparedata.prepareinputdata()
    output_data = preparedata.prepareoutputdata()

    model_param_config = config_manager.get_model_param_config()
    building_model = ModelBuilding(model_param_config)
    autoencoder = building_model.build_autoencoder()

    model_train_config = config_manager.get_model_training_config()
    training_model = TrainModel(model_train_config)
    training_model.train_model(input_data, output_data, autoencoder)
except Exception as e:
    raise e
    

[2024-09-24 14:39:33,021:INFO:util_functions:yaml file: ..\config\config.yaml loaded successfully]
[2024-09-24 14:39:33,026:INFO:util_functions:yaml file: ..\params.yaml loaded successfully]
[2024-09-24 14:39:33,028:INFO:util_functions:yaml file: ..\secret\secrets.yaml loaded successfully]
[2024-09-24 14:39:33,030:INFO:3125536882:Converting Blur Image Data into Array for Model Input]
[2024-09-24 14:39:33,034:INFO:3125536882:Image Data Successfully Converted into numpy array]
[2024-09-24 14:39:33,035:INFO:3125536882:Converting Clean Image Data into Array for Model Output]
[2024-09-24 14:39:33,038:INFO:3125536882:Image Data Successfully Converted into numpy array]
[2024-09-24 14:39:33,039:INFO:334777223:The Input shape is (128, 128, 3)]
[2024-09-24 14:39:33,040:INFO:334777223:The kernel size is 3 ]
[2024-09-24 14:39:33,040:INFO:334777223:The latent vector Lenght is 256]
[2024-09-24 14:39:33,043:INFO:334777223:Building Encoder Part of the Model]
[2024-09-24 14:39:33,225:INFO:334777223:Bui